<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons Lizenzvertrag" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a><br />Dieses Werk von Marvin Kastner ist lizenziert unter einer <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Creative Commons Namensnennung 4.0 International Lizenz</a>.

# Boston Wohnungsgrundstueck-Preise

Hier wird exemplarisch gezeigt, wie scikit-learn für eine Aufgabe wie eine lineare Regression eingestetzt werden kann.
Dies heißt, wir suchen nach den passenden Gewichten $\beta_i$ für die folgende Gleichung:

$$ y=\beta _{0} + \beta _{1} \cdot x_{1} + \cdots + \beta _{p} \cdot x_{p} + \varepsilon $$

Zunächst laden wir einen Datensatz.
Der wird bereits bei scikit-learn mit ausgeliefert.
Hierzu schauen wir uns nun die näheren Informationen an.

In [ ]:
import sklearn.datasets
boston = sklearn.datasets.load_boston()
print(boston.DESCR)

Wir können auch direkt die Daten betrachten.
Nur meistens helfen einem all die Zahlen in nicht aufbereiteter Form wenig.

In [ ]:
boston

## Untersuche Zielwert

In der Ausgabe oben kann man kaum etwas erkennen.
Aber wie teuer sind die Häuser überhaupt so ungefähr?

Dafür importieren wir zunächst das Modul `matplotlib`, mit welchem man visualisieren kann.

In [ ]:
import matplotlib.pyplot as plt

Nun können wir die Verteilung der Hauspreise grafisch darstellen:

In [ ]:
plt.hist(boston.target)
plt.ylabel("Anzahl")
plt.xlabel("Hauspreis in 1000 US-\$")
plt.show()

Nun wollen wir uns die Statistiken dazu berechnen.
Dafür nutzen wir das Modul `statistics`.

In [ ]:
import statistics

print("Mittelwert: \t\t", statistics.mean(boston.target))
print("Median: \t\t", statistics.median(boston.target))
print("Standardabweichung: \t", statistics.stdev(boston.target))

## Untersuche Attribute

Mithilfe der Attribute, den weiteren Eigenschaften einer Nachbarschaft, möchten wir später den Preis des Hauses vorhersagen.
Dafür schauen wir uns nun zunächst an, wie die Eigenschaften im Datensatz verteilt sind.
Dafür wird ein Scatterplot eingesetzt.

In [ ]:
for feature_name, data_column in zip(boston.feature_names, boston.data.T):
    plt.plot(data_column, boston.target, "b.")
    plt.ylabel("Hauspreis in 1000 US-\$")
    plt.xlabel(feature_name)
    plt.show()

Weitere Informationen zu der Plot-Funktion findet man mithilfe des Fragezeichen-Operators:

In [ ]:
?plt.plot

Für einige der Attribute war ein Scatterplot definitiv die falsche Visualisierungsform.
Welche Art von Plots würde sich hier noch anbieten - insbesondere für die Variable CHAS?
Eine Liste von Plot-Befehlen ist [hier](https://matplotlib.org/api/pyplot_summary.html) zu finden.

In [ ]:
# hier gerne weitere Plots kreieren

## Passe Lineare Regression an

Für das Maschinelle Lernen wird ein bestehender Datensatz in einen Trainings- und einen Test-Datensatz aufgeteilt.
Mit dem Trainings-Datensatz passen wir unsere $\beta_i$-Werte in der folgenden Gleichung an:

$$ y=\beta _{0} + \beta _{1} \cdot x_{1} + \cdots + \beta _{p} \cdot x_{p} + \varepsilon $$

Mit dem Test-Datensatz erproben wir dann, wie gut wir nun diesen "vorhersagen" können.
Die Idee ist, dass man die Modelle immer mit bislang nicht betrachteten Daten überprüfen soll.

In [ ]:
import sklearn.model_selection

Der Datensatz wird nun in einen Trainings-Datensatz und einen Test-Datensatz aufgeteilt.
Der Trainings-Datensatz wird 67 % der insgesamt vorhandenen Daten enthalten, der Test-Datensatz 33 %.

In [ ]:
X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(
    boston.data, boston.target, test_size=0.33)

Nun könnten wir die Attribute X_train mit dem Zielwert Y_train genauso untersuchen, wie wir es bereits mit `boston.data` gemacht haben.
Allerdings ist der Split zufällig erfolgt und deswegen sollte sich eigentlich nichts grundlegend geändert haben.

Nun importieren wir das Lineare Modell als Lernalgorithmus und passen die Gewichte $\beta_i$ an den vorliegenden Datensatz an:

In [ ]:
import sklearn.linear_model
lm = sklearn.linear_model.LinearRegression()
lm.fit(X_train, Y_train)

print("\nIntercept:", f"{lm.intercept_:9.4f}\n")

print("Feature\t    Koeffizient\n-----------|-----------------")
for feature, coefficient in zip(boston.feature_names, lm.coef_):
    print("{:<10}".format(feature), f"{coefficient:9.4f}")

Das heißt, dass die Lineare Regressionsgleichung ungefähr so lautet:

$$ y = 25,19 
    + -0,12 \cdot x_{1} 
    + 0,04 \cdot x_{2}
    + \cdots
    + \varepsilon $$

Je nach Durchlauf variiert diese Gleichung, weil der Trainings-Datensatz durch den zufälligen Split immer ein wenig anders aussieht.

Nun können wir das Lineare Modell auch schon einsetzen, um Vorhersagen zu treffen:

In [ ]:
print("\nEingabe:")
[print("\t{:<10}".format(feature), attr) for feature, attr in zip(boston.feature_names, X_test[0])]
print()
ausgabe = lm.predict(X_test[0:1])[0]
print("Ausgabe: ", ausgabe, "\n")

Das heißt, wenn ich für ein Haus die Werte gesammelt habe, dann kann ich nun einen Preis berechnen.
In diesem Fall sind es `22.77`.
Weil es 1000 US-$ sind, muss die Zahl noch mal 1000 gerechnet werden, d. h. der Wert ist 22.770.

Aber kann man dem Modell vertrauen? Das muss nun noch untersucht werden.
Dafür lassen wir alle Zielwerte vom Test-Datensatz erstellen und vergleichen das Ergebnis mit dem tatsächlichen, bekannten Wert.

In [ ]:
Y_pred = lm.predict(X_test)

Nun müssen wir noch den vorhergesagten mit dem tatsächlichen Wert vergleichen.

In [ ]:
plt.scatter(Y_test, Y_pred)
plt.xlabel("Tatsächlicher Hauspreis in $1000 US-\$")
plt.ylabel("Vorhergesagter Hauspreis in $1000 US-\$")

plt.plot([0, 50], [0, 50], "r", label="perfekter Treffer")
plt.legend()

plt.show()

Nun können wir den Fehler des Modells berechnen.
Das bezeichnet die Differenz zwischen der Vorhersage und dem tatsächlichen Wert.

In [ ]:
error = Y_test - Y_pred
plt.hist(error)
plt.xlabel("Differenz zwischen dem Hauspreis des Modells und des Tatsächlichen in $1000 US-\$")
plt.show()

Nun noch ein paar Metriken, wie wir sie für einen wissenschaftlichen Bericht brauchen könnten.
Dort kann man Visualisierungen, wie sie oben zu sehen sind, nicht immer mit aufnehmen, weil der Platz nicht mehr reicht...

In [ ]:
import sklearn.metrics

Hier gibt es eine ganze Reihe von Metriken, die in verschiedenen Bereichen der Wissenschaft verwendet werden.
Ob die Metrik im aktuellen Kontext hilfreich ist, muss der Mensch vor dem Bildschirm entscheiden.

In [ ]:
for score in dir(sklearn.metrics):
    if score.endswith("_score"):
        print(score)

Eine Auswahl von Metriken:

In [ ]:
print("Mean absolute error: %.2f" % sklearn.metrics.mean_absolute_error(Y_test, Y_pred))
print("Mean squared error : %.2f"  % sklearn.metrics.mean_squared_error(Y_test, Y_pred))
print("r^2                : %.2f"  % sklearn.metrics.r2_score(Y_test, Y_pred))